In [1]:
%matplotlib inline
import utils#; reload(utils)
from utils import *
from __future__ import division, print_function


import os
import random

import numpy as np
import matplotlib.pyplot as plt

from glob import iglob, glob

from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import load_img, img_to_array

from scipy.misc import imresize, toimage


ROOT_DIR = os.getcwd()
DATA_HOME_DIR = ROOT_DIR + '/data/'
%matplotlib inline



WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
Using Theano backend.


In [3]:
DATA_HOME_DIR
path = DATA_HOME_DIR+"fishnofish/"
batch_size=32
target_size = (360,640)

In [4]:
#path = "data/fish/sample/"



(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

raw_filenames = [f.split('/')[-1] for f in filenames]
raw_test_filenames = [f.split('/')[-1] for f in test_filenames]
raw_val_filenames = [f.split('/')[-1] for f in val_filenames]

# raw_val_filenames[0]

Found 3327 images belonging to 2 classes.
Found 450 images belonging to 2 classes.
Found 12153 images belonging to 1 classes.


In [12]:
for i,f in enumerate(raw_test_filenames):
    raw_test_filenames[i] = "test_stg2/"+raw_test_filenames[i] 

In [20]:
save_array("data/stg2_raw_test_filenames.dat",raw_test_filenames)

In [45]:
# trn = load_array(path+'results/trn_640.dat')
# val = load_array(path+'results/val_640.dat')
# test = load_array(path+'results/test_640.dat')

In [28]:
# conv_val_feat = load_array(DATA_HOME_DIR+'results/conv_val_640.dat')
# conv_trn_feat = load_array(DATA_HOME_DIR+'results/conv_trn_640.dat')
# conv_test_feat = load_array(DATA_HOME_DIR+'results/conv_test_640.dat')

In [16]:
# vgg640 = Vgg16BN((360, 640)).model
# vgg640.pop()
# print (vgg640.input_shape, vgg640.output_shape)
# vgg640.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])

# conv_layers,_ = split_at(vgg640, Convolution2D)

(None, 3, 360, 640) (None, 512, 22, 40)


In [4]:
train_datagen = image.ImageDataGenerator(
        shear_range=0.1,
        zoom_range=0.1,
        rotation_range=7.,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True)


# set shuffle=False if pre-computing

trn_flow = train_datagen.flow_from_directory(path+'train', target_size=target_size,
           class_mode='categorical', shuffle=True, batch_size=batch_size)


val_flow = image.ImageDataGenerator().flow_from_directory(path+"valid", target_size=target_size, batch_size=batch_size,
                                                   class_mode='categorical', shuffle=True)



Found 3327 images belonging to 2 classes.
Found 450 images belonging to 2 classes.


#### precomputing augmented vgg features 

Memory issues, skipping this and doing a full net

In [47]:
# da_conv_feat = vgg640.predict_generator(aug_trn_batches, aug_trn_batches.nb_sample*5)

# save_array(path+'results/aug_trn_feat1.dat', da_conv_feat)
# da_conv_feat = load_array(path+'results/aug_trn_feat1.dat')
#da_conv_feat = da_conv_feat[0:aug_trn_batches.nb_sample*3]

In [18]:

#Let's include the real training data as well in its non-augmented form.
# da_conv_feat = np.concatenate([da_conv_feat, conv_trn_feat])

#Since we've now got a dataset 6x bigger than before, we'll need to copy our labels 6 times too.
#da_trn_labels = np.concatenate([trn_labels]*3)


In [11]:
nf=128; p=0.

# from keras.layers.pooling import GlobalMaxPooling2D
def get_lrg_layers():
    return [
        BatchNormalization(axis=1, input_shape=conv_layers[-1].output_shape[1:]),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D((1,2)),
        Convolution2D(2,3,3, border_mode='same'), #fish/no fish
        Dropout(p),
        GlobalAveragePooling2D(), #change to Max
#         GlobalMaxPooling2D(),
        Activation('softmax')
    ]

In [12]:
model = Vgg16BN(target_size).model #(360, 640)
model.pop()

for layer in model.layers: #Freeze feature layers
    layer.trainable = False
    
    
    
conv_layers,_ = split_at(model, Convolution2D)

for l in get_lrg_layers():
    model.add(l)
    
print (model.input_shape, model.output_shape)

early_stopping  = keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)


model.compile(Adam(lr=0.001), 'categorical_crossentropy', metrics=['accuracy'])


/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


th
(None, 3, 360, 640) (None, 2)


In [10]:
# if precomputing 

#model = Sequential(get_lrg_layers())
#model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()

In [11]:
model.fit_generator(trn_flow, samples_per_epoch=trn_flow.nb_sample, 
                    nb_epoch=5,
                    validation_data=val_flow, nb_val_samples=val_flow.nb_sample,
                    callbacks=[early_stopping])

Epoch 1/5
3327/3327 [==============================] - 484s - loss: 0.1581 - acc: 0.9480 - val_loss: 0.3058 - val_acc: 0.9200
Epoch 2/5
3327/3327 [==============================] - 482s - loss: 0.0843 - acc: 0.9735 - val_loss: 0.2508 - val_acc: 0.9467
Epoch 3/5
3327/3327 [==============================] - 482s - loss: 0.0562 - acc: 0.9814 - val_loss: 0.1019 - val_acc: 0.9800
Epoch 4/5
3327/3327 [==============================] - 482s - loss: 0.0434 - acc: 0.9844 - val_loss: 0.1868 - val_acc: 0.9533
Epoch 5/5
3327/3327 [==============================] - 482s - loss: 0.0368 - acc: 0.9862 - val_loss: 0.0800 - val_acc: 0.9778


#### i'll use these weights for submission:

In [15]:
# model.save_weights(path+"models/fcn_aug_1e3_5epc_val_acc_0.9778.hd5")
model.load_weights(path+"models/fcn_aug_1e3_5epc_val_acc_0.9778.hd5")



In [14]:
model.optimizer.lr = 1e-4
model.fit_generator(trn_flow, samples_per_epoch=trn_flow.nb_sample, 
                    nb_epoch=5,
                    validation_data=val_flow, nb_val_samples=val_flow.nb_sample,
                    callbacks=[early_stopping])

Epoch 1/5
3327/3327 [==============================] - 483s - loss: 0.0286 - acc: 0.9898 - val_loss: 0.1028 - val_acc: 0.9756
Epoch 2/5
3327/3327 [==============================] - 482s - loss: 0.0274 - acc: 0.9922 - val_loss: 0.1634 - val_acc: 0.9711
Epoch 3/5
3327/3327 [==============================] - 481s - loss: 0.0194 - acc: 0.9943 - val_loss: 0.1656 - val_acc: 0.9689


In [22]:
# model.save_weights(path+"models/fcn_aug_1e4_3epc_val_acc_0.9778.hd5") #should have been 0.9689
model.load_weights(path+"models/fcn_aug_1e4_3epc_val_acc_0.9778.hd5") #should have been 0.9689 #ignore

#### Making last 2 layer of VGG trainable (ignore)

In [23]:
# model.layers[-18:-10]

In [17]:
for layer in model.layers[-18:-14]:
    layer.trainable = True     #didn't really improve results

In [19]:
model.optimizer.lr = 1e-6

model_fn = path+"models/" + 'fcn_aug_vggtunetop2_{val_acc:.2f}_{val_acc:.2f}val_acc-{val_loss:.2f}-loss_{epoch}epoch_ie6.h5' 
ckpt = ModelCheckpoint(filepath=model_fn, monitor='val_loss',
                           save_best_only=True, save_weights_only=True)

model.fit_generator(trn_flow, samples_per_epoch=trn_flow.nb_sample, 
                    nb_epoch=3,
                    validation_data=val_flow, nb_val_samples=val_flow.nb_sample,
                    callbacks=[early_stopping,ckpt])

Epoch 1/3
3327/3327 [==============================] - 509s - loss: 0.0286 - acc: 0.9907 - val_loss: 0.1824 - val_acc: 0.9644
Epoch 2/3
3327/3327 [==============================] - 505s - loss: 0.0238 - acc: 0.9940 - val_loss: 0.2324 - val_acc: 0.9533
Epoch 3/3
3327/3327 [==============================] - 505s - loss: 0.0282 - acc: 0.9916 - val_loss: 0.2169 - val_acc: 0.9622


In [20]:
model.optimizer.lr = 1e-7

model_fn = path+"models/" + 'fcn_aug_vggtunetop2_{val_acc:.2f}_{val_acc:.2f}val_acc-{val_loss:.2f}-loss_{epoch}epoch_1e7.h5' 
ckpt = ModelCheckpoint(filepath=model_fn, monitor='val_loss',
                           save_best_only=True, save_weights_only=True)

model.fit_generator(trn_flow, samples_per_epoch=trn_flow.nb_sample, 
                    nb_epoch=3,
                    validation_data=val_flow, nb_val_samples=val_flow.nb_sample,
                    callbacks=[early_stopping,ckpt])

Epoch 1/3
3327/3327 [==============================] - 508s - loss: 0.0185 - acc: 0.9937 - val_loss: 0.1836 - val_acc: 0.9644
Epoch 2/3
3327/3327 [==============================] - 507s - loss: 0.0247 - acc: 0.9925 - val_loss: 0.2014 - val_acc: 0.9644
Epoch 3/3
3327/3327 [==============================] - 507s - loss: 0.0249 - acc: 0.9925 - val_loss: 0.1551 - val_acc: 0.9689


In [13]:
# model.save_weights(path+"models/fcn_aug_vggtunetop2_1e5_3epc_val_acc_0.9844.h5")
# model.load_weights(path+"models/fcn_aug_vggtunetop2_1e5_3epc_val_acc_0.9844.h5") >> doesn't exist! 

### Making predictions 
With test augmentation


In [17]:
test_datagen = train_datagen
num_iterations = 5
preds = np.zeros((len(test_filenames), 2)) #1000 test images, fish or no fish probs 


for i in range(num_iterations):
    test_gen = test_datagen.flow_from_directory(path+"test", target_size=target_size, batch_size=batch_size,
                                                        class_mode=None, shuffle=False)
    preds += model.predict_generator(test_gen, val_samples=len(test_filenames))
    
    print ("finished predicting round {}".format(i+1))
    save_array(path+"results/stg2_pred_fullconv_aug_no_pseudo_part{}.dat".format(i+1),preds)
    
averaged_preds = preds / num_iterations  
# save_array(path+"results/pred_stg1_fullconv_aug_no_pseudo.dat",averaged_preds)
save_array(path+"results/stg2_pred_fullconv_aug_no_pseudo.dat",averaged_preds)

Found 12153 images belonging to 1 classes.
finished predicting round 1
Found 12153 images belonging to 1 classes.
finished predicting round 2
Found 12153 images belonging to 1 classes.
finished predicting round 3
Found 12153 images belonging to 1 classes.
finished predicting round 4
Found 12153 images belonging to 1 classes.
finished predicting round 5


###  Pseudo labeling 
Using test and validation predictions to strenghten the model 

wrongly implemented, and most probably overfits to training set, ignoring the results in this section

In [96]:
class MixIterator(object):
    def __init__(self, iters):
        self.iters = iters
        self.multi = type(iters) is list
#         if self.multi:
#             self.N = sum([it[0].N for it in self.iters])
#         else:
        self.N = sum([it.N for it in self.iters])

    def reset(self):
        for it in self.iters: it.reset()

    def __iter__(self):
        return self

    def next(self, *args, **kwargs):
#         if self.multi:
#             nexts = [[next(it) for it in o] for o in self.iters]
#             n0s = np.concatenate([n[0] for n in o])
#             n1s = np.concatenate([n[1] for n in o])
#             return (n0, n1)
#         else:
        nexts = [next(it) for it in self.iters]
        n0 = np.concatenate([n[0] for n in nexts])
        n1 = np.concatenate([n[1] for n in nexts])
        return (n0, n1)



In [91]:
# trn = load_array(path+'results/trn_640.dat')
# val = load_array(path+'results/val_640.dat')
# test = load_array(path+'results/test_640.dat')


In [97]:

# gen = image.DataGenerator()
# test_flow = train_datagen.flow_from_directory(path+"test", target_size=target_size, batch_size=batch_size,
#                                                         class_mode=None, shuffle=False)


trn_batch = train_datagen.flow_from_directory(path+'train', target_size=target_size,
           class_mode='categorical', shuffle=True, batch_size=22) #67% of 32

val_batch = train_datagen.flow_from_directory(path+"valid", target_size=target_size, batch_size=2,
                                                   class_mode='categorical', shuffle=True)

#i'm actually making a mistake to use validation set results, by right, i should be using predictions of the model as labels 
#but i'm making a concious choice to train on full training set

gen = image.ImageDataGenerator()

# train_datagen
# trn_batch  = gen.flow(trn,trn_labels,batch_size=22)
# val_batch  = gen.flow(val,val_labels,batch_size=2)

# test_batch = train_datagen.flow(test,averaged_preds,batch_size=8)

#using ensumbled weights instead
test_batch = train_datagen.flow(test,final_stg1_weights,batch_size=8,shuffle=False)






Found 3327 images belonging to 2 classes.
Found 450 images belonging to 2 classes.


In [98]:
mi = MixIterator([trn_batch, val_batch,test_batch])
# 

In [99]:
model.optimizer.lr = 1e-5
model.fit_generator(mi, mi.N, 
                    nb_epoch=5,
                    validation_data=val_flow, nb_val_samples=val_flow.nb_sample,
                    callbacks=[early_stopping])

Epoch 1/5
4768/4777 [============================>.] - ETA: 1s - loss: 0.0638 - acc: 0.9847

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/engine/training.py:1462: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


4800/4777 [==============================] - 670s - loss: 0.0636 - acc: 0.9848 - val_loss: 0.0632 - val_acc: 0.9800
Epoch 2/5
4783/4777 [==============================] - 667s - loss: 0.0519 - acc: 0.9868 - val_loss: 0.0282 - val_acc: 0.9889
Epoch 3/5
4783/4777 [==============================] - 667s - loss: 0.0423 - acc: 0.9923 - val_loss: 0.0196 - val_acc: 0.9933
Epoch 4/5
4783/4777 [==============================] - 667s - loss: 0.0457 - acc: 0.9904 - val_loss: 0.0211 - val_acc: 0.9889
Epoch 5/5
4783/4777 [==============================] - 667s - loss: 0.0415 - acc: 0.9923 - val_loss: 0.0061 - val_acc: 1.0000


There's a risk of overfitting by including full validation set (and labels) during training, but i use the resutls anyway, 
because i trust the network has sufficient means not to overfit (global average pooling)



In [100]:
model.save_weights(path+"models/fcn_aug_pseduo_1e5_5epc_val_acc_1.0.hd5") 


#### Creating predictions for test set

In [102]:
test_datagen = train_datagen
num_iterations = 5
preds = np.zeros((1000, 2)) #1000 test images, fish or no fish probs 


for i in range(num_iterations):
    test_gen = test_datagen.flow_from_directory(path+"test", target_size=target_size, batch_size=batch_size,
                                                        class_mode=None, shuffle=False)
    preds += model.predict_generator(test_gen, val_samples=1000)
    print ("finished predicting round {}".format(i+1))
    
    
averaged_preds = preds / num_iterations  
save_array(path+"results/pred_stg1_fullconv_aug_WITH_pseudo.dat",averaged_preds)

Found 1000 images belonging to 1 classes.
finished predicting round 1
Found 1000 images belonging to 1 classes.
finished predicting round 2
Found 1000 images belonging to 1 classes.
finished predicting round 3
Found 1000 images belonging to 1 classes.
finished predicting round 4
Found 1000 images belonging to 1 classes.
finished predicting round 5


##  Fully Connected on top
This is one of the models i created in the beginning. 
Using the probs for bagging with the fully conv net model above, reuslts in 0.03 improvement in final score

I should have done training and test augmentation here, but it's too late for second stage. 

In [9]:
# trn = get_data(path+'train', (360,640))
# val = get_data(path+'valid', (360,640))


Found 3327 images belonging to 2 classes.
Found 450 images belonging to 2 classes.


In [ ]:
test = get_data(path+'test', (360,640))


Found 12153 images belonging to 1 classes.


In [ ]:
save_array(path+'results/stg2_test_640.dat', test)


In [4]:
# save_array(path+'results/trn_640.dat', trn)
# save_array(path+'results/val_640.dat', val)

# trn = load_array(path+'results/trn_640.dat')
# val = load_array(path+'results/val_640.dat')
test = load_array(path+'results/stg2_test_640.dat') #was val_640.dat i hope i've not messed up


IOError: [Errno 2] No such file or directory: '/home/ubuntu/fish/data/fishnofish/results/stg2_test_640.dat/meta/sizes'

In [5]:
trn.shape, val.shape

((3327, 3, 360, 640), (450, 3, 360, 640))

In [4]:
from vgg16bn import Vgg16BN
# # # model = vgg_ft_bn(2)
vgg640 = Vgg16BN(size=(360, 640)).model

vgg640.pop()
print (vgg640.input_shape, vgg640.output_shape)
vgg640.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])


/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


(None, 3, 360, 640) (None, 512, 22, 40)


In [21]:
# vgg640.summary()

In [26]:
# trn = load_array(path+'results/trn.dat')
# val = load_array(path+'results/val.dat')

# gen = image.ImageDataGenerator()
# model.compile(optimizer=Adam(1e-3),
#        loss='categorical_crossentropy', metrics=['accuracy'])

# # model.fit(trn, trn_labels, batch_size=batch_size, nb_epoch=3, validation_data=(val, val_labels))

# model.load_weights(path+'results/ft1.h5')


In [5]:
conv_layers,fc_layers = split_at(vgg640, Convolution2D)
conv_model = Sequential(conv_layers)

# conv_feat = conv_model.predict(trn)
# conv_val_feat = conv_model.predict(val)




In [12]:
save_array(path+'results/conv_val_feat_640.dat', conv_val_feat)
save_array(path+'results/conv_feat_640.dat', conv_feat)


In [ ]:
conv_test_feat = conv_model.predict(test)
save_array(path+'results/stg2_conv_test_feat_640.dat', conv_test_feat)


In [7]:

def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        BatchNormalization(axis=1),
        Dropout(p/4),
        Flatten(),
        Dense(1024, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(1024, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(2, activation='softmax')
    ]

In [8]:
# load 640 version instead .. 
# conv_feat = load_array(DATA_HOME_DIR+'results/conv_feat_640.dat')
# conv_val_feat = load_array(DATA_HOME_DIR+'results/conv_val_feat_640.dat')
# conv_test_feat = load_array(DATA_HOME_DIR+'results/conv_test_feat_640.dat')


In [9]:
p = 0.6
# bn_model = vgg_ft_bn(1)
# conv_layers,fc_layers = split_at(vgg640, Convolution2D)

bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


# bn_model = Sequential(get_bn_layers(p))
# bn_model.compile(Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [42]:
conv_model.input_shape,conv_model.output_shape

((None, 3, 360, 640), (None, 512, 22, 40))

In [25]:
# bn_model.summary()

In [44]:
print (conv_feat.shape,bn_model.input_shape)

bn_model.fit(conv_feat, to_categorical(trn_classes), batch_size=batch_size, nb_epoch=3, 
             validation_data=(conv_val_feat, to_categorical(val_classes)))

# bn_model.fit(conv_feat, trn_classes, batch_size=batch_size, nb_epoch=3, 
#              validation_data=(conv_val_feat, val_classes))

(3327, 512, 22, 40) (None, 512, 22, 40)
Train on 3327 samples, validate on 450 samples
Epoch 1/3
3327/3327 [==============================] - 25s - loss: 0.5895 - acc: 0.7463 - val_loss: 0.2273 - val_acc: 0.9356
Epoch 2/3
3327/3327 [==============================] - 25s - loss: 0.3039 - acc: 0.8783 - val_loss: 0.1702 - val_acc: 0.9600
Epoch 3/3
3327/3327 [==============================] - 25s - loss: 0.1797 - acc: 0.9351 - val_loss: 0.1246 - val_acc: 0.9800


In [45]:
bn_model.fit(conv_feat, to_categorical(trn_classes), batch_size=batch_size, nb_epoch=3, 
             validation_data=(conv_val_feat, to_categorical(val_classes)))

Train on 3327 samples, validate on 450 samples
Epoch 1/3
3327/3327 [==============================] - 25s - loss: 0.1345 - acc: 0.9492 - val_loss: 0.2301 - val_acc: 0.9644
Epoch 2/3
3327/3327 [==============================] - 25s - loss: 0.1195 - acc: 0.9600 - val_loss: 0.1605 - val_acc: 0.9711
Epoch 3/3
3327/3327 [==============================] - 25s - loss: 0.0888 - acc: 0.9723 - val_loss: 0.1726 - val_acc: 0.9711


In [57]:
# bn_model.optimizer.lr /= 10
bn_model.fit(conv_feat, to_categorical(trn_classes), batch_size=batch_size, nb_epoch=5, 
             validation_data=(conv_val_feat, to_categorical(val_classes)))

Train on 3327 samples, validate on 450 samples
Epoch 1/5
3327/3327 [==============================] - 25s - loss: 0.0386 - acc: 0.9889 - val_loss: 0.2028 - val_acc: 0.9711
Epoch 2/5
3327/3327 [==============================] - 25s - loss: 0.0442 - acc: 0.9874 - val_loss: 0.1861 - val_acc: 0.9733
Epoch 3/5
3327/3327 [==============================] - 25s - loss: 0.0348 - acc: 0.9904 - val_loss: 0.1734 - val_acc: 0.9778
Epoch 4/5
3327/3327 [==============================] - 25s - loss: 0.0317 - acc: 0.9913 - val_loss: 0.1816 - val_acc: 0.9756
Epoch 5/5
3327/3327 [==============================] - 25s - loss: 0.0244 - acc: 0.9913 - val_loss: 0.2086 - val_acc: 0.9778


In [10]:
# bn_model.save_weights(path+"models/640_loss_0.0472_acc_0.9856_val_loss_0.1373_val_acc_0.9778")
# bn_model.save_weights(path+"models/loss_0.0244_acc_0.9913_val_loss_0.2086_val_acc_0.9778")
bn_model.load_weights(path+"models/loss_0.0244_acc_0.9913_val_loss_0.2086_val_acc_0.9778")

In [ ]:
predictions = bn_model.predict(conv_test_feat)

In [ ]:
# save_array(path+"results/preds_640_val_acc_0.9778.dat",predictions)
# save_array(path+"results/preds_640_loss_0.0244_acc_0.9913_val_loss_0.2086_val_acc_0.9778.dat",predictions)
save_array(path+"results/stg2_pred_dense_noaug_no_pseudo.dat",predictions)

In [ ]:
predictions[:,0].mean(),predictions[:,1].mean()

0 is fish class .., 1 no_fish

##### Stage 2 predictions:


In [13]:
test_batches = image.ImageDataGenerator().flow_from_directory(path+"test", target_size=target_size,
                class_mode=None, shuffle=False, batch_size=32)

Found 12153 images belonging to 1 classes.


In [14]:
conv_test_feat = conv_model.predict_generator(test_batches, len(test_filenames))


In [15]:
stg2_predictions = bn_model.predict(conv_test_feat, batch_size=32)


In [18]:
save_array(path+"results/stg2_pred_dense_noaug_no_pseudo.dat",stg2_predictions)

### Ensembling FCN and fully connected 

In [5]:

# fully_connected_preds_stg1 = load_array(path+"results/preds_640_loss_0.0244_acc_0.9913_val_loss_0.2086_val_acc_0.9778.dat")
# fcn_preds_stg1 = load_array(path+"results/pred_stg1_fullconv_aug_no_pseudo.dat")


fully_connected_preds_stg2 = load_array(path+"results/stg2_pred_dense_noaug_no_pseudo.dat")
fcn_preds_stg2 = load_array(path+"results/stg2_pred_fullconv_aug_no_pseudo.dat")


In [6]:
# final_stg1_weights = fully_connected_preds_stg1 + fcn_preds_stg1
# final_stg1_weights /= 2.0

final_stg2_weights = fully_connected_preds_stg2 + fcn_preds_stg2
final_stg2_weights /= 2.0

In [7]:
final_stg2_weights[0:1]

array([[ 0.9577,  0.0423]])

In [8]:
# save_array(path+"results/final_stg1_weights_ensumble.dat", final_stg1_weights)
save_array(path+"results/fishfinal_stg2_weights_ensumble.dat", final_stg2_weights)

#this resulted in 0.03 improvement in LB

> Lookback >  use final_stg1_weights to do pseudo labeling on FCN (0.003 result improvement, not sure if training time and risk of overfitting is worth it)

In [105]:
#now calculate predictions again with results of pseudo labeling and using full validation set:

fcn_preds_stg1_WITH_pseudo = load_array(path+"results/pred_stg1_fullconv_aug_WITH_pseudo.dat")

final_stg1_weights = fcn_preds_stg1_WITH_pseudo + fully_connected_preds_stg1 + fcn_preds_stg1
final_stg1_weights /= 3.0

final_stg1_weights[0:1]


array([[  9.9981e-01,   1.9445e-04]])

In [106]:
save_array(path+"results/final_stg1_weights_ensumble_with_pseudo.dat", final_stg1_weights)